In [2]:
# !pip install ultralyt
# from ultralytics import YOLO
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
# import seaborn as sns
import re
import numpy as np

from zipfile import ZipFile

from os import listdir
from os.path import isfile, join

import shutil

In [ ]:
# unzipping RDD2022 file
with ZipFile("RDD2022.zip", 'r') as zObject: 
    zObject.extractall(path="RDD2022")

In [3]:
countries = ['China_Drone', 'China_MotorBike', 'Czech', 'India', 'Japan', 'Norway', 'United_States']

In [ ]:
# unzipping individual countries folders
for country in countries:
    with ZipFile("RDD2022/RDD2022_all_countries/" + country + ".zip", 'r') as zObject: 
        zObject.extractall(path="RDD2022/" + country)

In [10]:
# copy all images to right folder
for country in countries:
    shutil.copytree("/gpfs/home5/tkoene/RDD2022/" + country + "/"+ country + "/train/images", "/gpfs/home5/tkoene/RDD2022/" + country + "/data/images/train")

In [9]:
# remove these folders
# shutil.rmtree('RDD2022.Japan')
# shutil.rmtree("/gpfs/home5/tkoene/RDD2022/Norway/data/images/train")

In [5]:
# create function to get labels
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = float(bbx.find('xmin').text)
            ymin = float(bbx.find('ymin').text)
            xmax = float(bbx.find('xmax').text)
            ymax = float(bbx.find('ymax').text)
            label = member.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [6]:
for country in countries: 
    xml_to_csv("/gpfs/home5/tkoene/RDD2022/" + country + "/" + country + "/train/annotations/xmls").to_csv('RDD2022/' + country + '/labels_train.csv')

In [15]:
# set all labels in right format in the right folder
for country in countries: 
    annotations = pd.read_csv('RDD2022/' + country + '/labels_train.csv')
    annotations['x'] = (((annotations['xmax'] - annotations['xmin']) / 2) + annotations['xmin']) / annotations['width']
    annotations['y'] = (((annotations['ymax'] - annotations['ymin']) / 2) + annotations['ymin']) / annotations['height']
    annotations['rel_width'] = (annotations['xmax'] - annotations['xmin']) / annotations['width']
    annotations['rel_height'] = (annotations['ymax'] - annotations['ymin']) / annotations['height']
    
    annotations['class_int'] = annotations['class']
    annotations['class_int'].replace({
        'Block crack': 0,
        'D00': 1,
        'D01': 2,
        'D0w0': 3,
        'D10': 4,
        'D11': 5,
        'D20': 6,
        'D40': 7,
        'D43': 8,
        'D44': 9,
        'D50': 10,
        'Repair': 11, 
        None: 12}, inplace=True)
    # print(country, annotations['class_int'].value_counts())

    filenames = annotations.filename.unique()
    for file in filenames:
        labels = annotations[annotations['filename'] == file]
        name = 'RDD2022/' + country + '/data/labels/train/' + str(file).replace('.jpg', '.txt')

        np.savetxt(name, labels[['class_int', 'x', 'y', 'rel_width', 'rel_height']], fmt='%i %.6e %.6e %.6e %.6e')

China_Drone 1     1426
4     1263
11     769
6      293
7       86
0        3
Name: class_int, dtype: int64
China_MotorBike 1     2678
4     1096
6      641
11     277
7      235
Name: class_int, dtype: int64
Czech 1    988
4    399
7    197
6    161
Name: class_int, dtype: int64
India 7     3187
6     2021
1     1555
9     1062
2      179
4       68
8       57
5       45
10      28
3        1
Name: class_int, dtype: int64
Japan 6     6199
1     4049
9     3995
4     3979
10    3553
7     2243
8      736
Name: class_int, dtype: int64
Norway 1    8570
4    1730
6     468
7     461
Name: class_int, dtype: int64
United_States 1    6750
4    3295
6     834
7     135
Name: class_int, dtype: int64


In [14]:
annotations['class_int'].value_counts()

1    6750
4    3295
6     834
7     135
Name: class_int, dtype: int64

'/gpfs/home5/tkoene/RDD2022/Czech/data/images/train'

In [16]:
# import os

# print(os.listdir())